<a href="https://colab.research.google.com/github/clkim/IntroNNDLPart2/blob/main/nn_zero_to_hero_micrograd_lecture_first_half_roughly_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Illustrating how the `autograd` automatic differentiation engine works in PyTorch.

This Jupyter notebook is extracted from notebook:
[micrograd_lecture_first_half_roughly.ipynb](https://github.com/karpathy/nn-zero-to-hero/blob/master/lectures/micrograd/micrograd_lecture_first_half_roughly.ipynb)  
by Andrej Karpathy  
Neural Networks: Zero to Hero  
Lecture 1: The spelled-out intro to neural networks and backpropagation: **building micrograd**

# Setup code

In [2]:
class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other):
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  def __mul__(self, other):
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')

    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward

    return out

  def backward(self):

    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [3]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right

  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

# Example - one simple function

b = a + a

a = 3.0  
b = a + a = 6.0

Pretend there is a loss function, *bloss*  
bloss = b = a + a

$ \frac{\partial bloss}{\partial a}
= \frac{\partial bloss}{\partial b} (\frac{\partial b}{\partial a}) $  
$ \phantom{\frac{\partial bloss}{\partial a}}
= (\frac{\partial b}{\partial a}) \frac{\partial bloss}{\partial b} $

Heuristic:  
loss gradient wrt equation input (i.e. here: a)  
$ \phantom{\frac{\partial bloss}{\partial a}} = \; $
local gradient $\;$ * $\;$ loss gradient wrt equation output (i.e. here: b)

$ \frac{\partial bloss}{\partial a}
= \frac{\partial b}{\partial a}  
= (1 + 1) \; \frac{\partial bloss}{\partial b} = (2) * 1 = 2 $

# Automatic gradient evaluation

In [ ]:
a = Value(3.0, label='a')
b = a + a   ; b.label = 'b'

print("---- before b.backward() ----")
print(f"a = {a}\na.grad = {a.grad}\n")
print(f"b = {b}\nb.grad = {b.grad}\n")

b.backward()

print("---- after b.backward() ----")
print(f"a.grad = {a.grad}\n")
print(f"b.grad = {b.grad}\n")

draw_dot(b)

# Example - more simple functions

$ a = -2.0 $

$ b = 3.0 $

$ d = a * b = -6 $

$ e = a + b = 1 $

$ f = d * e = -6 \enspace $ let's say f is the loss function

$ \frac{∂f}{∂d} = \frac{∂f}{∂d} \frac{∂f}{∂f} = e \; 1 = e = 1 $

$ \frac{∂f}{∂e} = \frac{∂f}{∂e} \frac{∂f}{∂f} = d \; 1 = d = -6 $

$ \frac{∂f}{∂b}
= \frac{∂f}{∂d} \frac{∂d}{∂b} + \frac{∂f}{∂e} \frac{∂e}{∂b}
= \frac{∂d}{∂b} \frac{∂f}{∂d} + \frac{∂e}{∂b} \frac{∂f}{∂e}
= (a \; e) + (1 \; d) = (-2) + (-6) = -8 $

$ \frac{∂f}{∂a}
= \frac{∂f}{∂d} \frac{∂d}{∂a} + \frac{∂f}{∂e} \frac{∂e}{∂a}
= \frac{∂d}{∂a} \frac{∂f}{∂d} + \frac{∂e}{∂a} \frac{∂f}{∂e}
= (b \; e) + (1 \; d)  = (3) + (-6) = -3 $

# Automatic gradient evaluation

In [ ]:
a = Value(-2.0, label='a')
b = Value(3.0, label='b')
d = a * b    ; d.label = 'd'
e = a + b    ; e.label = 'e'
f = d * e    ; f.label = 'f'

f.backward()

print(f"a.grad after f.backward() call = {a.grad}\n")
print(f"b.grad after f.backward() call = {b.grad}\n")
print(f"d.grad after f.backward() call = {d.grad}\n")
print(f"e.grad after f.backward() call = {e.grad}\n")
print(f"f.grad after f.backward() call = {f.grad}\n")

draw_dot(f)

# backward()

b.backward()

```
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()
```

$ self = b $

$ self.grad = 1.0 \; \rightarrow \; b.grad = 1.0  \; \rightarrow \; \frac{∂b}{∂b} = 1.0 $

# (a + a) expression

b = a + a
```
  def __add__(self, other):
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out
```
$ self = a, \enspace other = a $

$ b = out \; \rightarrow \; b.data = a.data + a.data, \enspace b.\_prev = (a, a), $

$ \enspace \enspace \enspace \enspace \enspace \enspace \enspace \enspace \enspace \,
b.\_backward = $
```
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
```

# _backward()

When b._backward() is called:

`self.grad += 1.0 * out.grad`

a.grad += 1.0 * b.grad

$ \frac{∂b}{∂a} \mathrel{+}= 1.0 * \frac{∂b}{∂b} = 1.0 * 1.0 = 1 $

`other.grad += 1.0 * out.grad`

a.grad += 1.0 * b.grad

$ \frac{∂b}{∂a} \mathrel{+}= 1.0 * \frac{∂b}{∂b} = 1.0 * 1.0 = 1 $

So  
a.grad = $ \frac{∂b}{∂a} = 1 + 1 = 2 $

SGD step:  
$ a_{new} = a_{old} - \eta \frac{∂b}{∂a} $